In [21]:
# Standard Python libraries
import os  # For operating system dependent functionality
import random  # For generating random numbers

# Data handling and manipulation libraries
import numpy as np  # For numerical operations on arrays

# PIL - Python Imaging Library
from PIL import Image  # For image processing

# PyTorch - Deep learning framework
import torch  # Main PyTorch library
import torch.nn as nn  # For building neural network layers
import torch.optim as optim  # For optimization algorithms
from torch.utils.data import DataLoader, Dataset, random_split  # Utilities for data handling in PyTorch

# torchvision - Utilities for working with image data and pretrained models
from torchvision import transforms  # For image transformations
from torchvision.models import resnet152, ResNet152_Weights  # Pretrained models and their weights

# tqdm - Library for progress bars
from tqdm import tqdm  # For displaying progress bars in loops



In [16]:
def setup_device():
    """Set up the PyTorch device - use GPU if available, otherwise CPU."""
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(f"Current device is : {device}")
    return device

# Call the function to set up the device
device = setup_device()



Current device is : cuda


In [13]:
# Define a transformation pipeline for preprocessing the image data
transform = transforms.Compose(
    [
        transforms.Resize((224, 224)),  # Resize images to 224x224 pixels
        transforms.RandomHorizontalFlip(), # Randomly flip images horizontally
        transforms.RandomRotation(10), # Randomly rotate images by up to 10 degrees
        transforms.ColorJitter(brightness=0.2, contrast=0.2), # Randomly change brightness and contrast
        transforms.RandomAffine(degrees=0, translate=(0.1, 0.1)),  # Affine transformation
        transforms.ToTensor(), # Convert images to PyTorch tensors
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]) # Normalize images with pre-defined mean and std
    ]
)

In [14]:
from torch.utils.data import Dataset
from PIL import Image

class CustomDataset(Dataset):
    def __init__(self, data, labels, transform=None):
        # Initialize dataset with data, labels, and optional transforms
        self.data = data
        self.labels = labels
        self.transform = transform

    def __len__(self):
        # Return number of items in dataset
        return len(self.labels)

    def __getitem__(self, idx):
        # Get image and label by index

        # Reshape image data and convert to PIL Image
        image = self.data[:, idx].reshape(300, 300, 3)
        image = Image.fromarray(image)

        # Apply transform if provided
        if self.transform:
            image = self.transform(image)

        # Get label for the image
        label = self.labels[idx]

        return image, label

In [19]:
def load_dataset(filepath):
    """Load dataset from a .npy file."""
    return np.load(filepath)

def create_loaders(train_dataset, val_dataset, train_batch_size, val_batch_size):
    """Create data loaders for training and validation datasets."""
    train_loader = DataLoader(train_dataset, batch_size=train_batch_size, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=val_batch_size)
    return train_loader, val_loader

# File paths for the training dataset and labels
training_data_filepath = 'data_train.npy'
training_labels_filepath = 'labels_train.npy'

# Loading the data and labels
data = load_dataset(training_data_filepath)
labels = load_dataset(training_labels_filepath)

# Creating a custom dataset with predefined transformations
dataset = CustomDataset(data, labels, transform=transform)

# Splitting the dataset into training and validation sets
train_size = int(0.9 * len(dataset))  # 90% for training
val_size = len(dataset) - train_size  # Remaining for validation

train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

# Creating data loaders
train_loader, val_loader = create_loaders(train_dataset, val_dataset, 128, 128)


In [20]:
import os
import torch
import torch.optim as optim
import torch.nn as nn
from torchvision.models import resnet152, ResNet152_Weights
from tqdm import tqdm

# Define paths and constants
SAVE_PATH = 'model saves/Best Model 6'  
NUM_CLASSES = 9
NUM_EPOCHS = 20
LEARNING_RATE = 0.0001
WEIGHT_DECAY = 1e-05
CONFIDENCE_THRESHOLD = 0.7

# Create directory for saving model
os.makedirs(SAVE_PATH, exist_ok=True)

def modify_resnet152(num_classes):
    model = resnet152(weights=ResNet152_Weights.DEFAULT)
    model.fc = nn.Linear(model.fc.in_features, num_classes)
    return model

def predict_with_confidence_threshold(outputs, threshold):
    probabilities = torch.nn.functional.softmax(outputs, dim=1)
    max_probs, preds = torch.max(probabilities, dim=1)
    preds[max_probs < threshold] = -1
    return preds

def train_model(model, criterion, optimizer, num_epochs, train_loader, val_loader, save_path):
    last_accuracy = 0
    for epoch in range(num_epochs):
        # Training phase
        model.train()
        train_loss, train_correct, train_total, _ = run_phase(model, train_loader, criterion, optimizer, training=True)

        # Evaluation phase
        model.eval()
        val_loss, val_correct, val_total, val_unknown = run_phase(model, val_loader, criterion, optimizer, training=False)

        # Print and save model
        print_epoch_results(epoch, num_epochs, train_loss, train_total, train_correct, val_loss, val_total, val_correct, val_unknown)
        if val_total > 0 and 100 * val_correct / val_total > last_accuracy:
            torch.save(model.state_dict(), os.path.join(save_path, f'model_epoch_{epoch+1}.pth'))
            last_accuracy = 100 * val_correct / val_total

def run_phase(model, loader, criterion, optimizer, training=True):
    total_loss, correct, total, unknown = 0.0, 0, 0, 0
    for images, labels in tqdm(loader):
        images, labels = images.to(device).float(), labels.to(device).long()  # Ensuring correct data types
        outputs = model(images)
        loss = criterion(outputs, labels)

        if training:
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

        total_loss += loss.item()
        if training:
            preds = torch.max(outputs.data, 1)[1]
        else:
            preds = predict_with_confidence_threshold(outputs, CONFIDENCE_THRESHOLD)
            unknown += (preds == -1).sum().item()
            preds, labels = preds[preds != -1], labels[preds != -1]

        total += labels.size(0)
        correct += (preds == labels).sum().item()

    return total_loss, correct, total, unknown

def print_epoch_results(epoch, num_epochs, train_loss, train_total, train_correct, val_loss, val_total, val_correct, val_unknown):
    train_accuracy = 100 * train_correct / train_total if train_total > 0 else 0
    val_accuracy = 100 * val_correct / val_total if val_total > 0 else 0
    print(f"Epoch [{epoch+1}/{num_epochs}]: Train Loss: {train_loss/len(train_loader):.4f}, "
          f"Train Acc: {train_accuracy:.2f}%, Val Loss: {val_loss/len(val_loader):.4f}, "
          f"Val Acc: {val_accuracy:.2f}%, Unknown Predictions: {val_unknown}")

# Main execution
model = modify_resnet152(NUM_CLASSES)
model.to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE, weight_decay=WEIGHT_DECAY)

train_model(model, criterion, optimizer, NUM_EPOCHS, train_loader, val_loader, SAVE_PATH)


100%|██████████| 7/7 [00:02<00:00,  2.55it/s]


Epoch [1/20]: Train Loss: 0.8411, Train Acc: 76.18%, Val Loss: 0.1248, Val Acc: 98.26%, Unknown Predictions: 40


100%|██████████| 7/7 [00:02<00:00,  2.59it/s]


Epoch [2/20]: Train Loss: 0.0760, Train Acc: 97.80%, Val Loss: 0.0993, Val Acc: 98.79%, Unknown Predictions: 16


100%|██████████| 7/7 [00:02<00:00,  2.61it/s]


...

Epoch [18/20]: Train Loss: 0.0071, Train Acc: 99.80%, Val Loss: 0.0739, Val Acc: 99.05%, Unknown Predictions: 2


100%|██████████| 7/7 [00:02<00:00,  2.62it/s]


Epoch [19/20]: Train Loss: 0.0115, Train Acc: 99.66%, Val Loss: 0.0687, Val Acc: 99.17%, Unknown Predictions: 5


100%|██████████| 7/7 [00:02<00:00,  2.59it/s]

Epoch [20/20]: Train Loss: 0.0050, Train Acc: 99.87%, Val Loss: 0.0621, Val Acc: 99.29%, Unknown Predictions: 5
